In [2]:
import tensorflow as tf
import numpy as np

In [7]:
class ResidualLayer(tf.keras.layers.Layer):
    """
    非常简单的残差层
    """
    def __init__(self,units,*args,**kwargs):
        super(ResidualLayer,self).__init__(*args,**kwargs)
        self.dense1=tf.keras.layers.Dense(units=units,activation=tf.nn.relu)
        self.dense2=tf.keras.layers.Dense(units=units,activation=tf.nn.relu)

    def call(self, inputs, **kwargs):
        outputs=self.dense1(inputs)
        outputs=self.dense2(outputs)
        return outputs+inputs

class DeepCrossing(tf.keras.Model):
    """
    deep crossing model，非常简单
    """
    def __init__(self,emb_input_output_dims,dense_feature_dim,res_layer_num,first_layer_units,*args,**kwargs):
        """

        :param emb_input_output_dims: [(int,int)] 用于构建多个embedding layer，每个tuple对应一个emb layer的参数
        :param dense_feature_dim: int 不需要embedding的特征的维度数目，需要这个值是因为res layer的构建需要这个值
        :param res_layer_num: int 后面跟多少层的res layer
        :param first_layer_units: int 最终的scoring layer的units
        :param args:
        :param kwargs:
        """
        super(DeepCrossing,self).__init__(*args,**kwargs)
        self.emb_layers=list()
        emb_output_dim_sum=0
        for input_dim,output_dim in emb_input_output_dims:
            self.emb_layers.append(tf.keras.layers.Embedding(input_dim=input_dim,output_dim=output_dim))
            emb_output_dim_sum+=output_dim
        self.res_layers=list()
        for _ in range(res_layer_num):
            self.res_layers.append(ResidualLayer(units=emb_output_dim_sum+dense_feature_dim))

        self.scoring_layer=tf.keras.layers.Dense(units=first_layer_units)

    def call(self, inputs,emb_indices,dense_index_bounds, training=None, mask=None):
        """

        :param inputs: tensor or np.ndarray shape如[batch_size,inputs_feature_dim]
        :param emb_indices: [int] 如[1,2,0]，代表inputs[:,1]输入到第0个emb layer，inputs[:,2]输入到第1个emb layer
        :param dense_index_bounds: [int,int]，代表inputs[:,int1:int2]是dense特征，不需要embedding
        :param training:
        :param mask:
        :return:
        """
        outputs=list()
        for i,emb_index in enumerate(emb_indices):
            outputs.append(self.emb_layers[i](inputs[:,emb_index]))
        dense_features=inputs[:,dense_index_bounds[0]:dense_index_bounds[1]]
        outputs.append(dense_features)
        outputs=tf.concat(outputs,axis=1)

        for res_layer in self.res_layers:
            outputs=res_layer(outputs)

        outputs=self.scoring_layer(outputs)
        return outputs

In [9]:
input_arr=np.concatenate([np.random.randint(0,5,size=[10,2]),np.random.random(size=[10,10])],axis=1).astype(np.float32)

deep_crossing=DeepCrossing(emb_input_output_dims=[(5,2),(5,3)],dense_feature_dim=10,res_layer_num=2,first_layer_units=1)

deep_crossing(input_arr,emb_indices=[0,1],dense_index_bounds=[2,12])

<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[-0.05615745, -0.39201617],
       [-0.03101929,  0.11204635],
       [ 0.68100584, -0.5315722 ],
       [ 0.745041  ,  0.2756614 ],
       [ 0.36445537, -0.69298005],
       [ 0.6602211 , -0.07050404],
       [ 0.27042696, -0.15288569],
       [ 0.11031098,  0.1377054 ],
       [ 0.3945523 , -0.73828685],
       [ 0.19533552,  0.07399537]], dtype=float32)>